In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [69]:
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2016-17.csv"
relative_string_map = "./Data_Indicator/String_map.csv"
relative_semantic = "./Data_Indicator/Semantic_map.csv"
relative_sem_threshold = "./Data_Indicator/Semantic_threshold_map.csv"
relative_rank = "./Data_Indicator/Ranking.csv"
relative_rank_unroll = "./Data_Indicator/Ranking_unroll.csv"
relative_tab_filter = "./Data_Indicator/Ranking_tab_filter.csv"
relative_chap_filter = "./Data_Indicator/Ranking_chap_filter.csv"
relative_final_map = "./Mapping_Indicator_2016-17.csv"
relative_count = "./Data_Indicator/Comparison_count.csv"


# Get the absolute path by joining the current directory with the relative path
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_rank =os.path.normpath( os.path.join(os.getcwd(), relative_rank))
absolute_path_rank_unroll =os.path.normpath( os.path.join(os.getcwd(), relative_rank_unroll))
absolute_path_tab_filter =os.path.normpath( os.path.join(os.getcwd(), relative_tab_filter))
absolute_path_chap_filter =os.path.normpath( os.path.join(os.getcwd(), relative_chap_filter))
absolute_path_final_map =os.path.normpath( os.path.join(os.getcwd(), relative_final_map))
absolute_path_count =os.path.normpath( os.path.join(os.getcwd(), relative_count))

In [9]:
def process_result(result_sdg, sdg_df, chapter_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['Rank']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        indicator_id = row['Indicator_id']
        
        for attr in attr_id:
            rows.append({
                'Attr_id': attr,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id,
                'top_n_indicator_id': indicator_id
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    indicator_name = dict(zip(sdg_df['Indicator_id'], sdg_df['Tentative Indicators']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    all_goals['Tentative Indicators'] = all_goals['top_n_indicator_id'].map(indicator_name.get)
    chapter_id = dict(zip(chapter_df['Attr_id'], chapter_df['Chapter_id']))
    chapter_name = dict(zip(chapter_df['Chapter_id'], chapter_df['Chapter_name']))
    table_id = dict(zip(chapter_df['Attr_id'], chapter_df['Table_id']))
    table_name = dict(zip(chapter_df['Table_id'], chapter_df['Table_name']))
    description = dict(zip(chapter_df['Attr_id'], chapter_df['Description']))
    all_goals['Chapter_id'] = all_goals['Attr_id'].map(chapter_id.get)
    all_goals['Chapter_name'] = all_goals['Chapter_id'].map(chapter_name.get)
    all_goals['Table_id'] = all_goals['Attr_id'].map(table_id.get)
    all_goals['Table_name'] = all_goals['Table_id'].map(table_name.get)
    all_goals['Description'] = all_goals['Attr_id'].map(description.get) 
    
                
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'Tentative Indicators', 'top_n_goal_id', 'top_n_target_id', 'top_n_indicator_id']]

    return all_goals


In [10]:
# Function to combine and remove duplicates from lists while maintaining order
def combine_lists_BC(row):
    list_B = row['top_n_attr_B']
    list_C = row['top_n_attr_C']
    
    # Combine lists and remove duplicates while maintaining order
    items = list(OrderedDict.fromkeys(list_B + list_C))
    return items

In [11]:
# Function to find intersection while maintaining order
def find_intersection_ABC(row):
    intersection = [x for x in row['top_n_attr_A'] if x in row['B_plus_C']]
    return intersection

In [12]:
def find_difference_A(row):
    top_n_attr = row['top_n_attr_A']
    intersection = row['A_and_BC']
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [13]:
# Function to find intersection while maintaining order
def find_intersection_BC(row):
    intersection = [x for x in row['top_n_attr_B'] if x in row['top_n_attr_C']]
    return intersection

In [14]:
def find_difference_B(row):
    top_n_attr = row['top_n_attr_B']
    intersection = set(row['A'] + row['B_and_C'] + row['A_and_BC'])
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [15]:
def find_difference_C(row):
    top_n_attr = row['top_n_attr_C']
    intersection = set(row['A'] + row['B_and_C'] + row['A_and_BC'] + row['B_minus_all'])
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [16]:
def final_rank(row):
    #combined_top_n_attr = sorted(set(list_B + list_C), key=lambda x: (list_B + list_C).index(x))
    items = row['A_and_BC'] + row['A'] + row['B_and_C'] + row['B_minus_all'] + row['C_minus_all']
    item_set = list(OrderedDict.fromkeys(items))    
    return item_set

In [17]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [18]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [19]:
sdg.drop(['Nodal Department','Other Related Major Departments'],axis=1,inplace=True)

In [20]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [21]:
sdg.reset_index(inplace = True, drop = True)

In [22]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [23]:
data_new = data.copy()
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,Nada Offices
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,Va Circles
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,Hoblies
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,Grama Panchayaths
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,No.of Taluks
...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total"
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total"
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total"
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Total"


In [24]:
sdg_string = pd.read_csv(absolute_path_string, sep=";", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_string

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"['population', 'international', 'poverty', 'li...",proportion of the population below the interna...,197,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.1...",['Districtwise Population By Age Groups And Se...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, ...","['Area and Population', 'Area and Population',...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, ...","['Male,0-14, Districtwise Population By Age Gr..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"['population', 'living', 'national', 'poverty'...",proportion of the population living below the ...,130,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.1...",['Districtwise Population By Age Groups And Se...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, ...","['Area and Population', 'Area and Population',...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, ...","['Male,0-14, Districtwise Population By Age Gr..."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"['men', 'woman', 'child', 'age', 'living', 'po...",proportion of men women and children of all ag...,94,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[10.8, 10.8, 10.8, 10.8, 10.8, 10.8, 13.1, 13....",['No .of Children in the age group 6 to 14 yea...,"[10, 10, 10, 10, 10, 10, 13, 13, 13, 13, 13, 1...","['Education', 'Education', 'Education', 'Educa...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","['Boys,No.of Children Enrolment age group betw..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"['population', 'covered', 'social', 'protectio...",percentage of the population covered by social...,263,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, ...",['Child Population 0-6 and proportion by resid...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","['Area and Population', 'Area and Population',...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","['Total,Child Population 0-6 Years 2011, Child..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"['population', 'living', 'household', 'access'...",proportion of the population living in househo...,187,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...",['Population and percentage share to total Pop...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","['Area and Population', 'Area and Population',...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Geographical Area Sq.Kms, Population and per..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"['amount', 'approved', 'funding', 'promote', '...",total amount of approved funding to promote th...,56,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[13.1, 13.1, 13.1, 13.1, 13.1, 13.2, 13.2, 13....",['Stree Shakti Self Help Groups and Members Ye...,"[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","['Women & Child Development', 'Women & Child D...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...

In [25]:
sdg_sim = pd.read_csv(absolute_path_semantic, sep=";", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_sim

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"['population', 'international', 'poverty', 'li...",proportion of the population below the interna...,"[['poverty', 'line'], ['population'], ['age', ...",[[-0.075216 0.47459 0.263355 ... -0.1...,"[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 26, 2...","[0.8120515928906179, 1.0000000000000009, 0.794...",54,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","['Geographical Area Sq.Kms, Population and per...","['Area and Population', 'Area and Population',...",['Population and percentage share to total Pop...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"['population', 'living', 'national', 'poverty'...",proportion of the population living below the ...,"[['living', 'national'], ['poverty', 'line'], ...",[[-0.219163 0.36556406 0.0332955 ... 0.1...,"[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 25, 2...","[0.6348859380215051, 0.8120515928906179, 0.794...",50,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","['Geographical Area Sq.Kms, Population and per...","['Area and Population', 'Area and Population',...",['Population and percentage share to total Pop...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"['men', 'woman', 'child', 'age', 'living', 'po...",proportion of men women and children of all ag...,"[['according', 'national'], ['children', 'ages...",[[-0.173088 0.43849001 0.0401875 ... 0.1...,"[511, 872, 873, 874, 875, 956, 25, 26, 27, 28,...","[0.5844719405576027, 0.6779166506008225, 0.885...",57,"[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[9, 14, 14, 14, 14, 17, 1, 1, 1, 1, 1, 1, 1, 1...","[9.2, 14.3, 14.3, 14.3, 14.3, 17.2, 1.6, 1.6, ...","['National Highway,P.w.d. Road Length in Km, P...","['TRANSPORT AND COMMUNICATION', 'Rural Develop...","['P.w.d. Road Length on 31-03-2017', 'Physical..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"['population', 'covered', 'social', 'protectio...",percentage of the population covered by social...,"[['age', 'persons'], ['children'], ['populatio...",[[-0.562305 0.03955675 0.20651501 ... 0.0...,"[715, 716, 717, 718, 719, 720, 721, 722, 723, ...","[0.8303263734472492, 0.7450329169830372, 0.835...",56,"[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 2, 2,...","[12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12....","['Nos,Boys,Government- Pre-Matric, Scheduled C...","['Social Welfare', 'Social Welfare', 'Social W...",['Scheduled Caste Student Hostels As on 31-3-2...
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"['population', 'living', 'household', 'access'...",proportion of the population living in househo...,"[['basic', 'services'], ['population'], ['livi...",[[-0.65325001 -0.41144501 -0.0148 ... -0.4...,"[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 515, ...","[0.6746278273013793, 1.0000000000000009, 0.586...",57,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 9, 9, 9, 9, 9, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","['Geographical Area Sq.Kms, Population and per..."

In [26]:
sdg_threshold = pd.read_csv(absolute_path_sem_threshold, dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, sep=";", converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_threshold

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"['population', 'international', 'poverty', 'li...",proportion of the population below the interna...,"[['poverty', 'line'], ['population'], ['age', ...",[[-0.075216 0.47459 0.263355 ... -0.1...,"[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...",386,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","['Geographical Area Sq.Kms, Population and per...","['Area and Population', 'Area and Population',...",['Population and percentage share to total Pop...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"['population', 'living', 'national', 'poverty'...",proportion of the population living below the ...,"[['living', 'national'], ['poverty', 'line'], ...",[[-0.219163 0.36556406 0.0332955 ... 0.1...,"[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...",135,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","['Geographical Area Sq.Kms, Population and per...","['Area and Population', 'Area and Population',...",['Population and percentage share to total Pop...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"['men', 'woman', 'child', 'age', 'living', 'po...",proportion of men women and children of all ag...,"[['according', 'national'], ['children', 'ages...",[[-0.173088 0.43849001 0.0401875 ... 0.1...,"[511, 872, 873, 874, 875, 956, 25, 26, 27, 28,...",307,"[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[9, 14, 14, 14, 14, 17, 1, 1, 1, 1, 1, 1, 1, 1...","[9.2, 14.3, 14.3, 14.3, 14.3, 17.2, 1.6, 1.6, ...","['National Highway,P.w.d. Road Length in Km, P...","['TRANSPORT AND COMMUNICATION', 'Rural Develop...","['P.w.d. Road Length on 31-03-2017', 'Physical..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"['population', 'covered', 'social', 'protectio...",percentage of the population covered by social...,"[['age', 'persons'], ['children'], ['populatio...",[[-0.562305 0.03955675 0.20651501 ... 0.0...,"[715, 716, 717, 718, 719, 720, 721, 722, 723, ...",339,"[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1...","[12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12....","['Nos,Boys,Government- Pre-Matric, Scheduled C...","['Social Welfare', 'Social Welfare', 'Social W...",['Scheduled Caste Student Hostels As on 31-3-2...
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"['population', 'living', 'household', 'access'...",proportion of the population living in househo...,"[['basic', 'services'], ['population'], ['livi...",[[-0.65325001 -0.41144501 -0.0148 ... -0.4...,"[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...",484,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","['Geographical Area Sq.Kms, Population and per...","['Area and Population', 'Area and Population',...",['Population and percentage share to total Pop...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, tran

In [27]:
result_sdg_new =sdg_string.copy()
result_sdg_sim_new =sdg_sim.copy()
result_sdg_threshold_new = sdg_threshold.copy()

In [28]:
result_sdg_new = result_sdg_new[['Goal No.', 'Target_id', 'Indicator_id', 'top_n_attr']]

In [29]:
result_sdg_new.head()

,Goal No.,Target_id,Indicator_id,top_n_attr
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126..."
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126..."
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839..."
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83...."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."


In [30]:
result_sdg_sim_new = result_sdg_sim_new[['Goal No.', 'Target_id', 'Indicator_id', 'top_n_attr']]

In [31]:
result_sdg_sim_new.head()

,Goal No.,Target_id,Indicator_id,top_n_attr
0,1,1.1,1.1.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
1,1,1.2,1.2.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
2,1,1.2,1.2.2,"[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28...."
3,1,1.3,1.3.1,"[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728..."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."


In [32]:
result_sdg_threshold_new = result_sdg_threshold_new[['Goal No.', 'Target_id', 'Indicator_id', 'top_n_attr']]

In [33]:
result_sdg_threshold_new.head()

,Goal No.,Target_id,Indicator_id,top_n_attr
0,1,1.1,1.1.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
1,1,1.2,1.2.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
2,1,1.2,1.2.2,"[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28...."
3,1,1.3,1.3.1,"[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728..."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."


In [34]:
result_sdg_new.rename(columns = {'top_n_attr':'top_n_attr_A'}, inplace=True)

In [35]:
result_sdg_new['top_n_attr_B'] = result_sdg_threshold_new.loc[:,'top_n_attr']
result_sdg_new['top_n_attr_C'] = result_sdg_sim_new.loc[:,'top_n_attr']

In [36]:
result_sdg_new

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28...."
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728..."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917..."
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ..."


In [37]:
# Apply the function to the merged dataframe
result_sdg_new['B_plus_C'] = result_sdg_new.apply(combine_lists_BC, axis=1)
result_sdg_new

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28...."
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728..."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
...,...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917..."
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ..."


In [38]:
# Apply the function to the merged dataframe
result_sdg_new['A_and_BC'] = result_sdg_new.apply(find_intersection_ABC, axis=1)
result_sdg_new

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126..."
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126..."
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839..."
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83...."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
...,...,...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845..."
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[533.0, 872.0]"
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844..."
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0]"


In [39]:
# Apply the function to the merged dataframe
result_sdg_new['A'] = result_sdg_new.apply(find_difference_A, axis=1)
result_sdg_new

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[]
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[]
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[76.0, 77.0, 78.0, 85.0, 86.0, 87.0, 88.0, 89...."
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[620.0, 621.0, 622.0, 623.0, 624.0, 625.0, 626..."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[872.0, 880.0, 881.0]"
...,...,...,...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[]
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[533.0, 872.0]",[]
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844...","[339.0, 446.0, 520.0]"
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0]",[]


In [40]:
# Apply the function to the merged dataframe
result_sdg_new['B_and_C'] = result_sdg_new.apply(find_intersection_BC, axis=1)
result_sdg_new

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[76.0, 77.0, 78.0, 85.0, 86.0, 87.0, 88.0, 89....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28...."
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[620.0, 621.0, 622.0, 623.0, 624.0, 625.0, 626...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728..."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[872.0, 880.0, 881.0]","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
...,...,...,...,...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[533.0, 872.0]",[],"[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917..."
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844...","[339.0, 446.0, 520.0]","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861..."
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0]",[],"[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ..."


In [41]:
# Apply the function to the merged dataframe
result_sdg_new['B_minus_all'] = result_sdg_new.apply(find_difference_B, axis=1)
result_sdg_new

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[720.0, 715.0, 719.0, 982.0, 983.0, 984.0, 985..."
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[714.0, 715.0, 719.0, 718.0, 720.0]"
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[76.0, 77.0, 78.0, 85.0, 86.0, 87.0, 88.0, 89....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[396.0, 399.0, 402.0, 405.0, 408.0, 411.0, 535..."
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[620.0, 621.0, 622.0, 623.0, 624.0, 625.0, 626...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[396.0, 399.0, 402.0, 405.0, 408.0, 411.0, 535..."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[872.0, 880.0, 881.0]","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[15.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23...."
...,...,...,...,...,...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[230.0, 231.0, 232.0, 233.0, 234.0, 235.0, 236..."
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[533.0, 872.0]",[],"[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[923.0, 924.0, 103.0, 104.0, 105.0, 106.0, 107..."
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844...","[339.0, 446.0, 520.0]","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[572.0, 573.0, 574.0, 591.0, 592.0, 593.0, 594..."
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0]",[],"[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165...."


In [42]:
# Apply the function to the merged dataframe
result_sdg_new['C_minus_all'] = result_sdg_new.apply(find_difference_C, axis=1)
result_sdg_new

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[720.0, 715.0, 719.0, 982.0, 983.0, 984.0, 985...",[]
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[714.0, 715.0, 719.0, 718.0, 720.0]",[]
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[76.0, 77.0, 78.0, 85.0, 86.0, 87.0, 88.0, 89....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[396.0, 399.0, 402.0, 405.0, 408.0, 411.0, 535...",[]
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[620.0, 621.0, 622.0, 623.0, 624.0, 625.0, 626...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[396.0, 399.0, 402.0, 405.0, 408.0, 411.0, 535...",[]
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[872.0, 880.0, 881.0]","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[15.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23....",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[230.0, 231.0, 232.0, 233.0, 234.0, 235.0, 236...",[]
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[533.0, 872.0]",[],"[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[923.0, 924.0, 103.0, 104.0, 105.0, 106.0, 107...",[]
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844...","[339.0, 446.0, 520.0]","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[572.0, 573.0, 574.0, 591.0, 592.0, 593.0, 594...",[]
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0]",[],"[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....",[]


In [43]:
result_sdg_new.columns

Index(['Goal No.', 'Target_id', 'Indicator_id', 'top_n_attr_A', 'top_n_attr_B',
       'top_n_attr_C', 'B_plus_C', 'A_and_BC', 'A', 'B_and_C', 'B_minus_all',
       'C_minus_all'],
      dtype='object')

In [44]:
# Apply the function to the merged dataframe
result_sdg_new['Rank'] = result_sdg_new.apply(final_rank, axis=1)
result_sdg_new

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all,Rank
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[720.0, 715.0, 719.0, 982.0, 983.0, 984.0, 985...",[],"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126..."
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[714.0, 715.0, 719.0, 718.0, 720.0]",[],"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126..."
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[76.0, 77.0, 78.0, 85.0, 86.0, 87.0, 88.0, 89....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[396.0, 399.0, 402.0, 405.0, 408.0, 411.0, 535...",[],"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839..."
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[620.0, 621.0, 622.0, 623.0, 624.0, 625.0, 626...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[396.0, 399.0, 402.0, 405.0, 408.0, 411.0, 535...",[],"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83...."
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[872.0, 880.0, 881.0]","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[15.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23....",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[230.0, 231.0, 232.0, 233.0, 234.0, 235.0, 236...",[],"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845..."
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[533.0, 872.0]",[],"[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[923.0, 924.0, 103.0, 104.0, 105.0, 106.0, 107...",[],"[533.0, 872.0, 898.0, 899.0, 900.0, 901.0, 917..."
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844...","[339.0, 446.0, 520.0]","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[572.0, 573.0, 574.0, 591.0, 592.0, 593.0, 594...",[],"[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844..."
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0]",[],"[27.0, 339.0, 384

In [45]:
final_df = result_sdg_new.copy()

In [46]:
final_df['Attr_count'] = final_df['Rank'].apply(lambda x: len(x))

In [47]:
final_df

,Goal No.,Target_id,Indicator_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all,Rank,Attr_count
0,1,1.1,1.1.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[720.0, 715.0, 719.0, 982.0, 983.0, 984.0, 985...",[],"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",386
1,1,1.2,1.2.1,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[714.0, 715.0, 719.0, 718.0, 720.0]",[],"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...",135
2,1,1.2,1.2.2,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[76.0, 77.0, 78.0, 85.0, 86.0, 87.0, 88.0, 89....","[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[396.0, 399.0, 402.0, 405.0, 408.0, 411.0, 535...",[],"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...",322
3,1,1.3,1.3.1,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[620.0, 621.0, 622.0, 623.0, 624.0, 625.0, 626...","[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[396.0, 399.0, 402.0, 405.0, 408.0, 411.0, 535...",[],"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....",358
4,1,1.4,1.4.1,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[872.0, 880.0, 881.0]","[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[15.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23....",[],"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....",487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,16.4,16.4.1,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",[],"[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[230.0, 231.0, 232.0, 233.0, 234.0, 235.0, 236...",[],"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...",458
175,16,16.5,16.5.1,"[533.0, 872.0]","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[533.0, 872.0]",[],"[872.0, 533.0, 898.0, 899.0, 900.0, 901.0, 917...","[923.0, 924.0, 103.0, 104.0, 105.0, 106.0, 107...",[],"[533.0, 872.0, 898.0, 899.0, 900.0, 901.0, 917...",75
176,17,17.1,17.1.1,"[339.0, 389.0, 446.0, 520.0, 839.0, 840.0, 841...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844...","[339.0, 446.0, 520.0]","[843.0, 844.0, 846.0, 848.0, 849.0, 852.0, 861...","[572.0, 573.0, 574.0, 591.0, 592.0, 593.0, 594...",[],"[389.0, 839.0, 840.0, 841.0, 842.0, 843.0, 844...",394
177,17,17.2,17.2.1,"[27.0, 339.0, 384.0, 386.0]","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.0, 339.0, 384.0, 386.0, 52.0, 53.0, 54.0, ...","[27.

In [48]:
final_df.to_csv(absolute_path_rank, sep=';', index=False)

In [49]:
all_goals = process_result(final_df, sdg, data)

In [50]:
all_goals.to_csv(absolute_path_rank_unroll, sep=';', index=False)

In [51]:
all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,Tentative Indicators,top_n_goal_id,top_n_target_id,top_n_indicator_id
0,120.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Male,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
1,121.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Female,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
2,122.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Total,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
3,123.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Male,15-59",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
4,124.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Female,15-59",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
50001,672.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Nos.,Primary Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50002,673.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Beds,Primary Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50003,674.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Nos.,Community Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50004,675.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Beds,Community Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2


In [52]:
# Group by 'col1' and count the distinct values in 'col2'
table_map = data_new.groupby('Chapter_id')['Table_id'].nunique().to_dict()
table_map

{'1': 10,
 '10': 15,
 '11': 6,
 '12': 6,
 '13': 6,
 '14': 5,
 '15': 7,
 '16': 3,
 '17': 3,
 '18': 1,
 '2': 11,
 '3': 2,
 '4': 18,
 '5': 4,
 '6': 5,
 '7': 2,
 '8': 4,
 '9': 4}

In [53]:
attr_map = data_new.groupby('Table_id')['Attr_id'].nunique().to_dict()
attr_map

{'1.1': 5,
 '1.10': 6,
 '1.2': 4,
 '1.3': 6,
 '1.4': 6,
 '1.5': 4,
 '1.6': 6,
 '1.7': 6,
 '1.8': 6,
 '1.9': 6,
 '10.1': 9,
 '10.10': 10,
 '10.11': 11,
 '10.12': 8,
 '10.13': 12,
 '10.14': 8,
 '10.15': 8,
 '10.2': 10,
 '10.3': 4,
 '10.4': 4,
 '10.5': 4,
 '10.6': 5,
 '10.7': 5,
 '10.8': 6,
 '10.9': 10,
 '11.1': 12,
 '11.2': 7,
 '11.3': 10,
 '11.4': 6,
 '11.5': 32,
 '11.6': 8,
 '12.1': 24,
 '12.2': 24,
 '12.3': 24,
 '12.4': 6,
 '12.5': 18,
 '12.6': 21,
 '13.1': 5,
 '13.2': 4,
 '13.3': 3,
 '13.4': 4,
 '13.5': 8,
 '13.6': 3,
 '14.1': 6,
 '14.2': 4,
 '14.3': 7,
 '14.4': 6,
 '14.5': 6,
 '15.1': 7,
 '15.2': 6,
 '15.3': 9,
 '15.4': 8,
 '15.5': 4,
 '15.6': 3,
 '15.7': 5,
 '16.1': 7,
 '16.2': 6,
 '16.3': 8,
 '17.1': 4,
 '17.2': 7,
 '17.3': 13,
 '18.1': 192,
 '2.1': 6,
 '2.11': 9,
 '2.12': 11,
 '2.2': 6,
 '2.3': 6,
 '2.4': 9,
 '2.5': 6,
 '2.6': 6,
 '2.7': 6,
 '2.8': 6,
 '2.9': 6,
 '3.1': 5,
 '3.2': 11,
 '4.1': 15,
 '4.10': 9,
 '4.11': 9,
 '4.12': 18,
 '4.13': 9,
 '4.14': 3,
 '4.15': 5,
 '4.16': 4,

In [54]:
all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,Tentative Indicators,top_n_goal_id,top_n_target_id,top_n_indicator_id
0,120.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Male,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
1,121.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Female,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
2,122.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Total,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
3,123.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Male,15-59",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
4,124.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Female,15-59",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
50001,672.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Nos.,Primary Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50002,673.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Beds,Primary Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50003,674.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Nos.,Community Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50004,675.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Beds,Community Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2


In [55]:
filter_df = all_goals.copy()

In [56]:
# Initialize an empty dictionary to store the counts
tab_result = {}
table_id_rm = {}
# Iterate over unique values in col1
for val in filter_df['top_n_indicator_id'].unique():
    # Filter the dataframe by the current value of col1
    filtered_df = filter_df[filter_df['top_n_indicator_id'] == val]
    
    # Count the occurrences of each value in col2 and store in a dictionary
    table_counts = filtered_df['Table_id'].value_counts().to_dict()
    
    table_id_rm[val] = {key:value for key, value in table_counts.items() if value < (attr_map[key]/2)}
    
    table_dict_new = {key:value for key, value in table_counts.items() if key not in table_id_rm[val]}
    
    if(len(table_dict_new) <= 4):
        # Sort the dictionary based on values in descending order
        sorted_dict = dict(sorted(table_counts.items(), key=lambda x: x[1], reverse=True))
        # Keep only the top 2 elements
        top_6_dict = dict(list(sorted_dict.items())[:6])
        tab_result[val] = top_6_dict
        
    else:
        # Add the col2 counts dictionary to the result dictionary
        tab_result[val] = table_dict_new

print(tab_result)

{'1.1.1': {'18.1': 168, '7.2': 12, '2.12': 11, '2.11': 9, '2.4': 9, '13.5': 8, '14.3': 7, '2.5': 6, '11.4': 6, '10.2': 6, '10.1': 6, '9.3': 6, '2.1': 6, '1.8': 6, '1.6': 6, '1.3': 6, '14.4': 6, '1.7': 6, '14.1': 6, '10.8': 6, '2.2': 6, '2.9': 6, '2.8': 6, '2.7': 6, '2.6': 6, '2.3': 6, '14.5': 6, '11.6': 5, '7.1': 5, '13.1': 5, '15.5': 4, '1.9': 4, '14.2': 4, '13.2': 3, '15.6': 3}, '1.2.1': {'7.2': 12, '2.12': 11, '2.11': 9, '2.4': 9, '2.2': 6, '2.9': 6, '2.8': 6, '2.7': 6, '1.6': 6, '2.6': 6, '2.1': 6, '10.8': 6, '2.5': 6, '2.3': 6, '1.8': 6, '1.7': 6, '13.1': 5, '11.6': 5, '14.3': 4, '13.2': 3}, '1.2.2': {'18.1': 174, '11.3': 10, '2.4': 9, '2.11': 9, '13.5': 8, '10.8': 6, '2.3': 6, '11.4': 6, '14.3': 6, '2.5': 6, '1.8': 6, '1.7': 6, '1.6': 6, '13.1': 5, '11.6': 5, '13.4': 4, '13.2': 4, '13.3': 3, '13.6': 3}, '1.3.1': {'12.2': 24, '12.3': 24, '12.1': 24, '12.6': 21, '12.5': 18, '2.12': 11, '11.3': 10, '2.4': 9, '2.11': 9, '13.5': 8, '10.13': 8, '17.2': 7, '2.8': 6, '1.6': 6, '1.7': 6, 

In [57]:
len(tab_result)

179

In [58]:
table_id_rm

{'1.1.1': {'17.2': 2, '11.5': 2, '6.3': 2, '4.1': 1, '1.2': 1, '17.3': 1},
 '1.2.1': {'17.2': 2, '14.1': 2, '9.2': 1},
 '1.2.2': {'11.5': 10,
  '6.1': 6,
  '10.2': 3,
  '10.1': 3,
  '17.2': 2,
  '14.1': 2,
  '10.14': 2,
  '10.15': 2,
  '10.11': 2,
  '15.3': 2,
  '9.2': 1,
  '2.1': 1,
  '1.4': 1,
  '5.1': 1,
  '10.6': 1,
  '10.7': 1},
 '1.3.1': {'18.1': 48,
  '11.5': 8,
  '6.1': 6,
  '14.3': 3,
  '10.1': 3,
  '10.2': 3,
  '14.1': 2,
  '10.11': 2,
  '10.14': 2,
  '10.15': 2,
  '1.4': 1,
  '10.6': 1,
  '10.7': 1},
 '1.4.1': {'14.3': 2, '6.4': 2, '14.2': 1, '9.1': 1, '9.4': 1},
 '1.5.1': {'18.1': 40,
  '9.1': 8,
  '11.5': 7,
  '6.1': 3,
  '11.6': 3,
  '4.18': 3,
  '6.4': 2,
  '8.4': 2,
  '13.1': 1,
  '13.4': 1,
  '15.5': 1,
  '15.7': 1,
  '16.2': 1,
  '16.3': 1,
  '5.1': 1,
  '4.3': 1,
  '15.4': 1},
 '1.6.1': {'18.1': 24,
  '10.13': 4,
  '10.11': 2,
  '5.4': 2,
  '8.3': 2,
  '15.2': 1,
  '9.2': 1,
  '6.3': 1,
  '4.3': 1,
  '4.1': 1},
 '2.1.1': {'16.2': 1,
  '9.3': 1,
  '11.6': 1,
  '4.17':

In [59]:
sum_ind=0
# Iterate over the outer dictionary
for key, inner_dict in table_id_rm.items():
    # Iterate over the inner dictionary
    for k, value in inner_dict.items():
        if k not in tab_result[key]:
            # Add the value to the result
            sum_ind += value

print(sum_ind)

4950


In [60]:
# List to store indices to remove
indices_to_remove = []
# Iterate over the rows of the DataFrame
for index, row in filter_df.iterrows():
    if row['top_n_indicator_id'] in table_id_rm:
        inner_dict = table_id_rm[row['top_n_indicator_id']]
        if row['Table_id'] in inner_dict:
            if row['Table_id'] not in tab_result[row['top_n_indicator_id']]:
                # Add index to remove list
                indices_to_remove.append(index)
            
indices_to_remove

[82,
 83,
 121,
 122,
 123,
 195,
 196,
 197,
 198,
 431,
 500,
 509,
 510,
 515,
 554,
 588,
 589,
 590,
 591,
 592,
 599,
 615,
 616,
 617,
 627,
 628,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 647,
 648,
 649,
 650,
 651,
 652,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 841,
 842,
 955,
 1079,
 1080,
 1081,
 1105,
 1106,
 1107,
 1108,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1132,
 1133,
 1134,
 1135,
 1136,
 1137,
 1138,
 1139,
 1140,
 1141,
 1142,
 1143,
 1144,
 1145,
 1146,
 1147,
 1148,
 1149,
 1150,
 1151,
 1152,
 1153,
 1154,
 1155,
 1156,
 1157,
 1158,
 1159,
 1160,
 1161,
 1162,
 1163,
 1164,
 1165,
 1166,
 1167,
 1168,
 1169,
 1170,
 1171,
 1172,
 1173,
 1174,
 1175,
 1176,
 1177,
 1178,
 1179,
 1180,
 1181,
 1182,
 1183,
 1184,
 1185,
 1186,
 1187,
 1188,
 1189,
 1190,
 1191,
 1192,
 1278,
 1279,
 1280,
 1385,
 1386,
 1387,
 1643,
 1688,
 1689,
 1690,
 1691,
 1692,
 1693,
 1694,
 1695,


In [61]:
len(indices_to_remove)

4950

In [62]:
# Remove rows using indices from indices_to_remove list
filter_df.drop(indices_to_remove, inplace=True)

filter_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,Tentative Indicators,top_n_goal_id,top_n_target_id,top_n_indicator_id
0,120.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Male,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
1,121.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Female,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
2,122.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Total,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
3,123.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Male,15-59",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
4,124.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Female,15-59",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
50000,671.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,No.of Pvt. Hospitals Including Nursing Homes C...,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50001,672.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Nos.,Primary Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50002,673.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Beds,Primary Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
50003,674.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Nos.,Community Health Centres","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2


In [63]:
filter_df.to_csv(absolute_path_tab_filter, sep=';', index=False)

In [64]:
tab_count_att = filter_df['top_n_indicator_id'].value_counts().to_dict()

In [65]:
filter_df_copy = filter_df.copy()

In [66]:
def filter_dataframe(df, n, chap_list):
    result_df = df.copy() # Initialize an empty DataFrame to store the filtered rows
    
    # Loop over each unique 'Target_id'
    for target_id, group in df.groupby('top_n_target_id'):
        num_rows = len(group)  # Count the number of rows for this 'Target_id'
        
        # Check if the number of rows is greater than the threshold 'n'
        if num_rows > n:
            idx_list = []
            drop_list = []
            for chap in chap_list:
                # Find the indices corresponding to chapters "2" and "3" for this 'Target_id'
                idx_chapter = group[group['Chapter_id'] == chap].index
                idx_list.append(idx_chapter)
                
            for item in idx_list:
                if len(item)>1:
                    total_rows_to_drop = item[len(item)//2:]
                    drop_list.append(total_rows_to_drop)
            
            for i in drop_list:
                if len(i)>0:
                    result_df = result_df.drop(i)
    result_df.reset_index(inplace = True, drop = True)
    return result_df

In [67]:
chap_ll = ['2', '4', '10', '12', '18']
# Filter the DataFrame using the function
final_map = filter_dataframe(filter_df_copy, 350, chap_ll)

In [70]:
final_map.to_csv(absolute_path_final_map, sep=';', index=False)

In [71]:
chapter_df = filter_df.copy()

In [72]:
# Initialize an empty dictionary to store the counts
chap_result = {}
chap_id_rm = {}

# Iterate over unique values in col1
for val in chapter_df['top_n_indicator_id'].unique():
    # Filter the dataframe by the current value of col1
    filtered_df = chapter_df[chapter_df['top_n_indicator_id'] == val]
    chap_list = []
    for tab in filtered_df['Table_id'].unique():
        chap_filter = filtered_df[filtered_df['Table_id'] == tab]
        
        # Count the occurrences of each value in col2 and store in a dictionary
        chap_counts = chap_filter['Chapter_id'].unique().tolist()
        
        chap_list.extend(chap_counts)
    chap_dict = pd.Series(chap_list).value_counts().to_dict()
    chap_id_rm[val] = {key:value for key, value in chap_dict.items() if value < (table_map[key]/2)}
    chap_dict_new = {key:value for key, value in chap_dict.items() if key not in chap_id_rm[val]}
    if(len(chap_dict_new) <= 1):
        # Sort the dictionary based on values in descending order
        sorted_dict = dict(sorted(chap_dict.items(), key=lambda x: x[1], reverse=True))
        # Keep only the top 2 elements
        top_4_dict = dict(list(sorted_dict.items())[:4])
        chap_result[val] = top_4_dict
 
    else:
        # Add the col2 counts dictionary to the result dictionary
        chap_result[val] = chap_dict_new

print(chap_result)

{'1.1.1': {'2': 11, '1': 5, '14': 5, '13': 3, '7': 2, '18': 1}, '1.2.1': {'2': 11, '7': 1}, '1.2.2': {'13': 6, '11': 3, '18': 1}, '1.3.1': {'2': 11, '13': 6, '12': 6}, '1.4.1': {'2': 11, '1': 10, '11': 6, '15': 4, '18': 1}, '1.5.1': {'11': 3, '17': 1, '5': 1, '1': 1}, '1.6.1': {'13': 6, '14': 5, '12': 4, '16': 3, '7': 2, '8': 2}, '2.1.1': {'2': 11, '1': 3, '11': 1, '17': 1}, '2.1.2': {'2': 11, '3': 2}, '2.2.1': {'11': 6, '13': 6, '14': 5}, '2.2.2': {'13': 6, '14': 5}, '2.3.1': {'4': 18, '8': 4, '18': 1, '7': 1}, '2.3.2': {'8': 4, '7': 2, '16': 2, '18': 1}, '2.4.1': {'2': 11, '13': 6, '14': 5, '8': 4, '11': 3, '18': 1}, '2.4.2': {'2': 11, '8': 4, '15': 4, '18': 1, '7': 1}, '2.4.3': {'8': 4, '18': 1}, '2.5.1': {'4': 12, '5': 2}, '2.5.2': {'4': 11, '1': 10, '3': 1, '18': 1}, '2.6.1': {'4': 18, '12': 4, '7': 1}, '2.7.1': {'6': 5, '8': 4, '16': 2, '18': 1}, '2.8.1': {'11': 6, '12': 6, '8': 4, '17': 2, '18': 1}, '2.8.2': {'14': 2, '1': 1, '10': 1, '5': 1}, '3.1.1': {'13': 6, '2': 3, '17': 1,

In [73]:
chap_id_rm

{'1.1.1': {'10': 3, '15': 2, '11': 2, '9': 1},
 '1.2.1': {'1': 3, '13': 2, '10': 1, '14': 1, '11': 1},
 '1.2.2': {'2': 4, '1': 3, '10': 1, '14': 1},
 '1.3.1': {'1': 3, '10': 2, '11': 2, '17': 1},
 '1.4.1': {'10': 2, '9': 1, '5': 1, '13': 1, '14': 1},
 '1.5.1': {'17': 1, '5': 1, '1': 1},
 '1.6.1': {'10': 4, '1': 3, '11': 1},
 '2.1.1': {'1': 3, '11': 1, '17': 1, '16': 1, '13': 1},
 '2.1.2': {'1': 3, '6': 2, '14': 2, '13': 1, '15': 1, '10': 1, '9': 1},
 '2.2.1': {'10': 4, '2': 3, '12': 1, '1': 1, '17': 1, '16': 1},
 '2.2.2': {'10': 4, '2': 2, '11': 1, '12': 1, '1': 1, '17': 1, '16': 1},
 '2.3.1': {'6': 2, '5': 1, '16': 1, '12': 1},
 '2.3.2': {'4': 7, '17': 1, '10': 1},
 '2.4.1': {'4': 4},
 '2.4.2': {'10': 3, '11': 1, '16': 1, '1': 1, '5': 1},
 '2.4.3': {'15': 3},
 '2.5.1': {'1': 3, '11': 2, '9': 1, '10': 1, '13': 1, '8': 1},
 '2.5.2': {'2': 5, '10': 4, '11': 2, '12': 1, '13': 1},
 '2.6.1': {'10': 4, '2': 3, '13': 1, '16': 1, '17': 1, '9': 1},
 '2.7.1': {'2': 3, '1': 1, '4': 1, '17': 1, '5

In [74]:
missing = []

# Iterate through the data dictionary
for key, value in chap_result.items():
    # Check if the sub-dictionary is empty
    if not value:
        # Add the key to the missing list
        missing.append(key)
missing

[]

In [75]:
# List to store indices to remove
chap_indices_to_remove = []
# Iterate over the rows of the DataFrame
for index, row in chapter_df.iterrows():
    if row['top_n_indicator_id'] in table_id_rm:
        inner_dict = chap_id_rm[row['top_n_indicator_id']]
        if row['Chapter_id'] in inner_dict:
            if row['top_n_indicator_id'] not in chap_result or row['Chapter_id'] not in chap_result[row['top_n_indicator_id']]:
                # Add index to remove list
                chap_indices_to_remove.append(index)

chap_indices_to_remove

[71,
 72,
 73,
 74,
 75,
 76,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 199,
 200,
 209,
 210,
 211,
 372,
 381,
 382,
 383,
 384,
 385,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 425,
 426,
 427,
 428,
 429,
 430,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 511,
 512,
 513,
 514,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 526,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 573,
 574,
 575,
 576,
 577,
 578,
 579,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 587,
 593,
 594,
 595,
 596,
 597,
 598,
 600,
 601,
 602,
 603,
 604,
 605,
 606,
 607,
 608,
 618,
 619,
 620,
 621,
 622,
 623,
 869,
 870,
 883,
 884,
 885,
 886,
 887,
 888,
 956,
 957,
 958,
 959,
 960,
 961,
 1082,
 1083,
 1084,
 1085,
 1086,
 1087,
 1088,
 1089,
 1090,
 

In [76]:
len(chap_indices_to_remove)

9476

In [77]:
# Remove rows using indices from indices_to_remove list
chapter_df.drop(chap_indices_to_remove, inplace=True)

chapter_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,Tentative Indicators,top_n_goal_id,top_n_target_id,top_n_indicator_id
0,120.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Male,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
1,121.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Female,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
2,122.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Total,0-14",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
3,123.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Male,15-59",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
4,124.0,2,Area and Population,2.11,Districtwise Population By Age Groups And Sex ...,"Female,15-59",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
49958,623.0,10,Education,10.13,No. of Medical Colleges and Students Medical C...,"Total,Students,Indian System of Medicine Colle...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
49959,629.0,10,Education,10.13,No. of Medical Colleges and Students Medical C...,"Students Intake,Medical Colleges Allopathy","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
49960,631.0,10,Education,10.13,No. of Medical Colleges and Students Medical C...,"Students Intake,Dental Colleges","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
49961,632.0,10,Education,10.14,No.of Polytechnic Colleges and Students Year- ...,"No.of Institutions,Polytechnic- Government","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2


In [78]:
chapter_df['top_n_indicator_id'].nunique()

179

In [79]:
chap_count_att = chapter_df['top_n_indicator_id'].value_counts().to_dict()

In [80]:
chap_count_att

{'5.6.2': 511,
 '14.6.1': 476,
 '12.5.1': 471,
 '4.7.1': 469,
 '15.9.2': 434,
 '1.4.1': 429,
 '15.6.1': 424,
 '2.8.1': 423,
 '3.10.1': 422,
 '2.3.1': 420,
 '14.8.1': 413,
 '12.3.1': 410,
 '14.9.1': 399,
 '14.4.1': 393,
 '11.2.1': 390,
 '2.4.1': 380,
 '3.1.2': 371,
 '2.5.2': 362,
 '16.4.1': 356,
 '14.2.1': 353,
 '15.5.1': 350,
 '9.8.1': 344,
 '3.7.2': 344,
 '2.4.2': 339,
 '10.3.1': 338,
 '1.1.1': 335,
 '8.11.1': 334,
 '11.3.1': 325,
 '5.7.1': 324,
 '10.4.1': 321,
 '8.8.2': 315,
 '17.1.1': 314,
 '14.11.1': 313,
 '11.1.1': 311,
 '11.4.1': 309,
 '4.5.1': 307,
 '12.4.1': 307,
 '11.8.1': 306,
 '15.4.1': 303,
 '8.4.1': 301,
 '4.8.1': 301,
 '6.7.1': 296,
 '4.2.2': 293,
 '4.9.1': 292,
 '11.7.1': 291,
 '4.4.1': 289,
 '14.7.2': 288,
 '14.12.1': 288,
 '2.7.1': 287,
 '15.3.2': 286,
 '12.1.1': 284,
 '14.4.2': 284,
 '11.6.1': 274,
 '15.1.3': 269,
 '14.1.1': 269,
 '14.3.1': 269,
 '14.2.2': 269,
 '6.8.1': 268,
 '14.10.1': 267,
 '5.5.1': 267,
 '15.2.3': 263,
 '7.1.1': 261,
 '15.4.2': 260,
 '11.10.1': 25

In [81]:
tab_count_att

{'5.6.2': 544,
 '4.7.1': 535,
 '12.5.1': 534,
 '14.6.1': 520,
 '14.8.1': 510,
 '4.5.1': 501,
 '2.3.1': 485,
 '3.10.1': 481,
 '1.4.1': 480,
 '15.6.1': 463,
 '2.8.1': 463,
 '15.9.2': 462,
 '9.8.1': 459,
 '14.2.1': 456,
 '2.5.2': 456,
 '16.4.1': 453,
 '12.3.1': 452,
 '14.9.1': 450,
 '11.4.1': 447,
 '3.7.2': 437,
 '2.4.1': 432,
 '15.5.1': 420,
 '11.2.1': 417,
 '3.1.2': 415,
 '2.4.2': 406,
 '14.4.1': 405,
 '12.4.1': 400,
 '8.11.1': 399,
 '9.1.2': 394,
 '11.7.1': 389,
 '11.6.2': 386,
 '11.6.1': 385,
 '14.2.2': 383,
 '6.6.1': 378,
 '1.1.1': 377,
 '14.4.2': 375,
 '17.1.1': 368,
 '10.4.1': 364,
 '15.3.2': 364,
 '8.8.2': 363,
 '11.3.1': 362,
 '4.8.1': 361,
 '12.1.1': 356,
 '10.3.1': 355,
 '15.4.1': 353,
 '17.2.1': 348,
 '14.7.2': 347,
 '14.12.1': 347,
 '5.1.1': 346,
 '14.1.1': 346,
 '5.7.1': 344,
 '11.1.1': 343,
 '14.3.1': 342,
 '4.2.2': 340,
 '2.3.2': 335,
 '3.3.3': 335,
 '15.1.3': 333,
 '6.7.1': 331,
 '8.4.1': 327,
 '17.2.2': 327,
 '2.7.1': 327,
 '14.11.1': 325,
 '11.9.1': 323,
 '11.8.1': 322,

In [82]:
rank_count_att = all_goals['top_n_indicator_id'].value_counts().to_dict()

In [83]:
rank_filter_count_att = final_map['top_n_indicator_id'].value_counts().to_dict()

In [84]:
count_att_list = [(key, value) for key, value in chap_count_att.items()]

count_att_df = pd.DataFrame(count_att_list, columns=['Indicator_id', 'Chap_count'])
count_att_df

,Indicator_id,Chap_count
0,5.6.2,511
1,14.6.1,476
2,12.5.1,471
3,4.7.1,469
4,15.9.2,434
...,...,...
174,6.4.1,35
175,8.10.1,29
176,2.8.2,27
177,14.5.1,18


In [85]:
count_att_df['Tab_count'] = count_att_df['Indicator_id'].map(tab_count_att)

In [86]:
count_att_df['Filter_count'] = count_att_df['Indicator_id'].map(rank_filter_count_att)

In [87]:
count_att_df['Rank_count'] = count_att_df['Indicator_id'].map(rank_count_att)

In [88]:
count_att_df

,Indicator_id,Chap_count,Tab_count,Filter_count,Rank_count
0,5.6.2,511,544,337,550
1,14.6.1,476,520,309,531
2,12.5.1,471,534,418,537
3,4.7.1,469,535,308,569
4,15.9.2,434,462,280,469
...,...,...,...,...,...
174,6.4.1,35,69,69,88
175,8.10.1,29,126,126,138
176,2.8.2,27,58,56,88
177,14.5.1,18,28,28,35


In [89]:
chapter_df.to_csv(absolute_path_chap_filter, sep=';', index=False)

In [90]:
count_att_df.to_csv(absolute_path_count, sep=';', index=False)